https://github.com/luispedro/BuildingMachineLearningSystemsWithPython/blob/55e4b80ac0234f2d45d0906edfbe0ab99f7a033e/ch02/seeds_knn_sklearn.py

# 밀 씨앗의 분류

In [10]:
# Basic imports
from __future__ import print_function
import numpy as np
from load import load_dataset

In [18]:
features, labels = load_dataset('seeds')

There are seven features that are present, which
are as follows:
- area A
- perimeter P
- compactness C = 4πA/P2
- length of kernel
- width of kernel
- asymmetry coefficient
- length of kernel groove

There are three classes(labels), corresponding to three wheat varieties:
- Canadian
- Koma
- Rosa

In [87]:
# Import sklearn implementation of KNN
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier(n_neighbors=4)
n = len(features)
correct = 0.0

In [23]:
for ei in range(n):
    # 210개의 'True'로 이루어진 array 생성
    training = np.ones(n, bool)
    
    # ei- 0~209까지 한번씩 0이 들어간다
    training[ei] = 0
    
    # testing에는 training에 *(-1)한 값들이 들어감
    testing = ~training
    
    # ei==1 일 때에는 training[1]을 제외한 값들이 fitting된다
    classifier.fit(features[training], labels[training])
    pred = classifier.predict(features[ei])
    correct += (pred == labels[ei])

/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

In [24]:
print('Result of leave-one-out: {}'.format(correct/n))

Result of leave-one-out: [ 1.75238095]


# Import KFold object

In [86]:
from sklearn.cross_validation import KFold

# means will hold the mean for each fold
means = []

# kf is a generator of pairs (training,testing) so that each iteration
# implements a separate fold.
kf = KFold(len(features), n_folds=3, shuffle=True)
for training,testing in kf:
    # We learn a model for this fold with `fit` and then apply it to the
    # testing data with `predict`:
    classifier.fit(features[training], labels[training])
    prediction = classifier.predict(features[testing])

    # np.mean on an array of booleans returns the fraction of correct decisions
    # for this fold:
    curmean = np.mean(prediction == labels[testing])
    means.append(curmean)
print('Result of cross-validation using KFold: {}'.format(means))

Result of cross-validation using KFold: [0.97142857142857142, 0.9285714285714286, 0.90000000000000002]


# The function cross_val_score does the same thing as the loop above with a single function call

In [78]:
from sklearn.cross_validation import cross_val_score

crossed = cross_val_score(classifier, features, labels)
print('Result of cross-validation using cross_val_score: {}'.format(crossed))

Result of cross-validation using cross_val_score: [ 0.97222222  0.94202899  0.85507246]


In [32]:
# The results above use the features as is, which we learned was not optimal
# except if the features happen to all be in the same scale. 
# We can pre-scale the features as explained in the main text:

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

classifier = Pipeline([('norm', StandardScaler()), ('knn', classifier)])
crossed = cross_val_score(classifier, features, labels)
print('Result with prescaling: {}'.format(crossed))

Result with prescaling: [ 0.94444444  0.92753623  0.84057971]


# Now, generate & print a cross-validated confusion matrix for the same result

In [90]:
from sklearn.metrics import confusion_matrix

names = list(set(labels))
labels = np.array([names.index(ell) for ell in labels])
preds = labels.copy()
preds[:] = -1
for train, test in kf:
    classifier.fit(features[train], labels[train])
    preds[test] = classifier.predict(features[test])

cmat = confusion_matrix(labels, preds)

print('Confusion matrix: [rows represent true outcome, columns predicted outcome]')
print(cmat)

Confusion matrix: [rows represent true outcome, columns predicted outcome]
[[62  4  4]
 [ 7 63  0]
 [ 7  0 63]]


In [74]:
# The explicit float() conversion is necessary in Python 2 (Otherwise, result is rounded to 0)
acc = cmat.trace()/float(cmat.sum())
print('Accuracy: {0:.1%}'.format(acc))

Accuracy: 93.8%
